In [2]:
!pip install -q tensorflow

In [5]:
# 1. Подключение Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Папки с данными
train_dir = '/content/dataset/train'
val_dir = '/content/dataset/val'

# 3. Импорты
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers

# 4. Параметры
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS_TOP = 15        # Сначала обучаем только верхушку
EPOCHS_FINE_TUNE = 40 # Потом дообучаем всю сеть

# 5. Загрузчики данных
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# 6. Модель
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base_model.trainable = False  # сначала замораживаем

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# 7. Компиляция и обучение только верхушки
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Обучаем только верхние слои...")
history_top = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_TOP
)

# 8. Разморозка EfficientNet
print("Размораживаем всю модель...")
base_model.trainable = True

# Обычно уменьшают скорость обучения после разморозки
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 9. Дообучение всей модели
print("Дообучаем всю модель...")
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_FINE_TUNE
)

# 10. Сохранение
model.save('/content/drive/MyDrive/weed_model_efficientnet.keras')
print("✅ Модель полностью обучена и сохранена!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 1676 images belonging to 5 classes.
Found 815 images belonging to 5 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Обучаем только верхние слои...
Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


105/105 ━━━━━━━━━━━━━━━━━━━━ 49s 267ms/step - accuracy: 0.2209 - loss: 1.6461 - val_accuracy: 0.3006 - val_loss: 1.6081
Epoch 2/15
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 66ms/step - accuracy: 0.2346 - loss: 1.6025 - val_accuracy: 0.1840 - val_loss: 1.6087
Epoch 3/15
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - accuracy: 0.2211 - loss: 1.6089 - val_accuracy: 0.3006 - val_loss: 1.6063
Epoch 4/15
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 58ms/step - accuracy: 0.2265 - loss: 1.6081 - val_accuracy: 0.3006 - val_loss: 1.6040
Epoch 5/15
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - accuracy: 0.2373 - loss: 1.6053 - val_accuracy: 0.3006 - val_loss: 1.6033
Epoch 6/15
105/105 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step - accuracy: 0.2447 - loss: 1.6037 - val_accuracy: 0.3006 - val_loss: 1.6023
Epoch 7/15
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 66ms/step - accuracy: 0.2372 - loss: 1.6046 - val_accuracy: 0.3006 - val_loss: 1.6014
Epoch 8/15
105/105 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step - accuracy: 0.2308 - loss: 1.6027 - val_accuracy: 0